In [70]:
import requests
import os
import json
import backoff

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {os.getenv('API_TOKEN')}"
}

#@backoff.on_exception(backoff.expo,
#                      requests.exceptions.RequestException,
#                      max_tries=10)
def call_get(url):
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # This will raise a HTTPError for bad responses (4xx and 5xx)
    #print(response.json())
    return response.json()

def get_latest_movie():

    url = "https://api.themoviedb.org/3/movie/latest"
    return call_get(url)


def get_movie_credits_by_id(id):
    url = f"https://api.themoviedb.org/3/movie/{id}/credits?language=en-US"
    try:
        return call_get(url)
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            try:
                error_response = e.response.json()
                if error_response.get('status_code') == 34:
                    print(f"No credits found for ID: {id}")
                    return None
                
            except ValueError:
                print(f"Received unexpected response: {e.response.text}")
        else:
            print(f"An error occurred: {e}")
            raise e


def get_movie_by_id(id):
    url = f"https://api.themoviedb.org/3/movie/{id}?language=en-US"
    try:
        return call_get(url)
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            try:
                error_response = e.response.json()
                if error_response.get('status_code') == 34:
                    print(f"No movie found for ID: {id}")
                    return None
                
            except ValueError:
                print(f"Received unexpected response: {e.response.text}")
        elif e.response.status_code == 429:
            print("Time out. Waiting for 10 seconds")
            #TODO: move it to call_get add time sleep
        else:
            print(f"An error occurred: {e}")
            raise e

def get_movie_from_omdb(imdb_id, api_key=os.getenv('OMDB_KEY')):
    url = f"https://www.omdbapi.com/?i={imdb_id}&apikey={api_key}"
    response = requests.get(url)
    response.raise_for_status()  # This will raise a HTTPError for bad responses (4xx and 5xx)
    return response.json()

def fetch_all_movies(start_id, last_id):
    all_movies = [] 
    for id in range(start_id, last_id + 1):  # assuming IDs start at 0
        movie = get_movie_by_id(id)
        if movie is not None and movie["imdb_id"]:
            # check this movie in omdb
            try: 
                omdb_json = get_movie_from_omdb(movie["imdb_id"])
                #print(omdb_json)
                merged = {**movie, **omdb_json}
            except requests.exceptions.HTTPError as e:
                print(f"OMDB Received response: {e.response.text}")
            all_movies.append(merged)
    return all_movies

def fetch_all_credits(start_id, last_id):
    all_credits = [] 
    for id in range(start_id, last_id + 1):  # assuming IDs start at 0
        credit = get_movie_credits_by_id(id)
        if credit is not None:
            all_credits.append(credit)
    return all_credits


In [87]:
import pandas as pd
#last_movie = get_latest_movie()['id']
last_movie = 3100
all_movies = fetch_all_movies(3001, last_movie)


pd.set_option('display.max_columns', None)
df = pd.json_normalize(all_movies)
df.to_parquet(f"tmdb/movies_{last_movie}.parquet", compression='gzip')
df.head(1)


No movie found for ID: 3005
No movie found for ID: 3018
No movie found for ID: 3020
No movie found for ID: 3038
No movie found for ID: 3039
No movie found for ID: 3044
No movie found for ID: 3045
No movie found for ID: 3046
No movie found for ID: 3047
No movie found for ID: 3057
No movie found for ID: 3058
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit reached!"}
OMDB Received response: {"Response":"False","Error":"Request limit r

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path
0,False,/lX5nHKBnF35dzTzYNavCBrZ6twU.jpg,NaN,6400000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 10749, 'name': 'Romance'}]",,3001,tt0060522,en,How to Steal a Million,A woman must steal a statue from a Paris museum to help conceal her father's art forgeries.,14.715,/xaf3pwmITJvfz9Ab8DiGM8OOtBC.jpg,"[{'id': 1286, 'logo_path': None, 'name': 'World Wide Productions', 'origin_country': ''}, {'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVRBLzaVmcp.png', 'name': '20th Century Fox', 'origin_country': 'US'}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1966-07-13,4400000,123,"[{'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'}]",Released,A movie about those who appreciate the finest things in life... for free!,How to Steal a Million,False,7.49,528,How to Steal a Million,1966,Approved,19 Aug 1966,123 min,"Comedy, Crime, Romance",William Wyler,"George Bradshaw, Harry Kurnitz","Audrey Hepburn, Peter O'Toole, Eli Wallach",The daughter of an art forger teams up with a burglar to steal one of her father's forgeries and protect his secret.,"English, French",United States,1 nomination,https://m.media-amazon.com/images/M/MV5BYTk4MTBjY2QtMjNiOS00MWZmLWE4MTUtMjc0NDA3YzE2ZTkxXkEyXkFqcGdeQXVyMTMxMTY0OTQ@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '100%'}]",N/A,7.5,"29,171",tt0060522,movie,01 Mar 2013,N/A,N/A,N/A,True,NaN,NaN,NaN,NaN,NaN


In [88]:
import pandas as pd
#last_movie = get_latest_movie()['id']
last_movie = 3000
all_movies = fetch_all_credits(1401, last_movie)


pd.set_option('display.max_columns', None)
df = pd.json_normalize(all_movies)
df.to_parquet(f"credits/credits_{last_movie}.parquet", compression='gzip')
df.head(1)

No credits found for ID: 1401
No credits found for ID: 1409
No credits found for ID: 1425
No credits found for ID: 1431
No credits found for ID: 1432
No credits found for ID: 1434
No credits found for ID: 1445
No credits found for ID: 1446
No credits found for ID: 1447
No credits found for ID: 1449
No credits found for ID: 1451
No credits found for ID: 1453
No credits found for ID: 1454
No credits found for ID: 1455
No credits found for ID: 1456
No credits found for ID: 1457
No credits found for ID: 1458
No credits found for ID: 1459
No credits found for ID: 1460
No credits found for ID: 1461
No credits found for ID: 1462
No credits found for ID: 1463
No credits found for ID: 1464
No credits found for ID: 1465
No credits found for ID: 1466
No credits found for ID: 1467
No credits found for ID: 1468
No credits found for ID: 1469
No credits found for ID: 1470
No credits found for ID: 1471
No credits found for ID: 1472
No credits found for ID: 1473
No credits found for ID: 1474
No credits

id  \
0  1402   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [45]:
pd.set_option("max_colwidth", None)
df = pd.read_parquet('credits/credits_1400.parquet')
#df.head(1)

In [63]:
import ast
pd.set_option("max_colwidth", None)
df = pd.read_parquet('tmdb/movies_100.parquet')
df.head(1)



,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path
0,False,/dQL2wJZo05GDd21VgOacMeCuyZy.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]",,2,tt0094675,fi,Ariel,"After the coal mine he works at closes and his father commits suicide, a Finnish man leaves for the city to make a living but there, he is framed and imprisoned for various crimes.",9.651,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,"[{'id': 2303, 'logo_path': None, 'name': 'Villealfa Filmproductions', 'origin_country': 'FI'}]","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1988-10-21,0,73,"[{'english_name': 'Finnish', 'iso_639_1': 'fi', 'name': 'suomi'}]",Released,,Ariel,False,7.1,262,Ariel,1988,Not Rated,01 Oct 1990,72 min,"Comedy, Crime, Romance",Aki Kaurismäki,Aki Kaurismäki,"Turo Pajala, Susanna Haavisto, Matti Pellonpää",A Finnish man goes to the city to find a job after the mine where he worked is closed and his father commits suicide.,Finnish,Finland,3 wins & 1 nomination,https://m.media-amazon.com/images/M/MV5BOGU5OGVlNjEtNTE3Ny00YWZkLThlMmQtYjlkNmNjNTA1OGY5XkEyXkFqcGdeQXVyMjI0MjMwMzQ@._V1_SX300.jpg,"[{'Source': 'Internet Movie Database', 'Value': '7.5/10'}]",N/A,7.5,"7,369",tt0094675,movie,N/A,N/A,N/A,N/A,True,NaN,None,None,None


In [68]:
data = df.copy()

def extract_rating(ratings, source):
    for rating in ratings:
        if rating['Source'] == source:
            return rating['Value']
    return None

data = data.drop(columns=['Title'])
data['Rotten_Tomatoes_Rating'] = data['Ratings'].apply(lambda x: extract_rating(x, 'Rotten Tomatoes')) 
data['Metacritic_Rating'] = data['Ratings'].apply(lambda x: extract_rating(x, 'Metacritic')) 
data['Internet_Movie_Database_Rating'] = data['Ratings'].apply(lambda x: extract_rating(x, 'Internet Movie Database')) 


# Standardize column names
data.columns = data.columns.str.lower().str.replace(' ', '_')
data = data.drop(columns=['backdrop_path', 'poster_path', 'video', 
                          'belongs_to_collection.name', 'belongs_to_collection.poster_path', 'belongs_to_collection.backdrop_path',
                          'belongs_to_collection.id','response'
                          , 'poster', 'homepage', 'imdbid'] )#, 'type', 'dvd', 'website'])



data['genres'] = data['genres'].apply(lambda x: ', '.join([d['name'] for d in x]))
data['production_companies'] = data['production_companies'].apply(lambda x: ', '.join([d['name'] for d in x]))
data['production_countries'] = data['production_countries'].apply(lambda x: ', '.join([d['iso_3166_1'] for d in x]))
data['spoken_languages'] = data['spoken_languages'].apply(lambda x: ', '.join([d['iso_639_1'] for d in x]))
data.head(20)

data.to_csv("data.csv")





#df.to_csv('movies.csv')

In [ ]:
# Remove duplicate columns
#data = df.drop(columns=['title'])

# Explode JSON columns
def explode_json(column):
    data[column] = data[column].apply(ast.literal_eval)
    return data.explode(column)

data = explode_json('genres')
data = explode_json('production_companies')

# Parse awards column for nomination information
def parse_nominations(awards_text):
    if 'nomination' in awards_text:
        return int(awards_text.split()[0])
    return 0

data['nomination_count'] = data['awards'].apply(parse_nominations)